# Spatial Models

Overview of today's topics:

  - quick refresher
  - spatial fixed effects
  - spatial regimes
  - spatial lag
  - spatial error
  - geographically-weighted regression
  
## 1. Quick refresher

### 1.1. Theory and models

**Spatial models** are models that include geographic information to account for spatial relationships and processes. They can take on many different forms:

  - Spatially-explicit regression models (with [PySAL](https://pysal.org))
  - Agent-based modeling and/or cellular automata (with [Mesa](https://mesa.readthedocs.io/))
  - Bayesian spatial models using Markov chain Monte Carlo methods (with [PyMC3](https://docs.pymc.io/))

We will focus on spatially-explicit regression models in this module. Spatially-explicit regression models are a type of **statistical model**: sets of assumptions plus mathematical relationships between variables, producing a formal representation of some theory. We are essentially trying to explain the process underlying the generation of our observed data. **Spatial inference** introduces explicit spatial relationships into the statistical modeling framework, as both theory-driven (e.g., spatial spillovers) and data-driven (e.g., MAUP) issues could otherwise violate modeling assumptions.

### 1.2. Statistical inference refresher

**Statistical inference** is the process of using a sample to *infer* the characteristics of an underlying population (from which this sample was drawn) through estimation and hypothesis testing. What is the probability distribution (the probabilities of occurrence of different possible outcome values of our response variable)? Contrast this with descriptive statistics, which focus simply on describing the characteristics of the sample itself.

Common goals of inferential statistics include:

  - parameter estimation and confidence intervals
  - hypothesis rejection
  - prediction and explanation
  - model selection

Schools of statistical inference:

  - frequentist
    - frequentists think of probability as proportion of time some outcome occurs (relative frequency)
    - given lots of repeated trials, how likely is the observed outcome?
    - concepts: statistical hypothesis testing, *p*-values, confidence intervals
  - bayesian
    - bayesians think of probability as amount of certainty observer has about an outcome occurring (subjective probability)
    - probability as a measure of how much info the observer has about the real world, updated as info changes
    - concepts: prior probability, likelihood, bayes' rule, posterior probability
    
### 1.3. Regression refresher

This course presumes you're already comfortable with multiple regression and OLS, as a prerequisite.

Regression assumptions:

  - an additive, linear relationship between response and predictors
  - uncorrelated predictors
  - uncorrelated, homoskedastic, normally-distributed errors
  
Regression topics:

  - specification: choosing variables to include and the functional form
  - transformation: pre-processing to improve the linear fit (log, power, etc)
  - estimation: using an algorithm (such as OLS, WLS, MLE, etc) to estimate (aka, fit or train) your model's parameters
  - validation and diagnostics: model's goodness of fit ($R^2$), parameters' statistical significance ($t$-test and $p$-values), check errors and assumptions (residual plot, Q-Q plot, etc), outlier influence (leverage), robustness checks (alternative specifications)
  - resampling: cross-validation (out-of-sample test with training/testing subsets) and bootstrapping (random subsampling to generate estimates' distribution)
  - model selection and regularization: bias-variance tradeoff (overfitting and underfitting), lasso (L1 regularization), ridge (L2 regularization), hyperparameters
  
## 2. Setup and data prep

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import pysal as ps

In [ ]:
# load CA tracts
tracts_ca = gpd.read_file('../../data/tl_2017_06_tract/').set_index('GEOID')

# keep LA, ventura, orange counties only (and drop offshore island tracts)
to_drop = ['06037599100', '06037599000', '06111980000', '06111990100', '06111003612']
tracts_ca = tracts_ca[tracts_ca['COUNTYFP'].isin(['037', '059', '111'])].drop(index=to_drop)

# project tracts
crs = '+proj=utm +zone=11 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
tracts_ca = tracts_ca.to_crs(crs)
tracts_ca.shape

In [ ]:
# load CA tract-level census variables
df_census = pd.read_csv('../../data/census_tracts_data_ca.csv', dtype={'GEOID10':str}).set_index('GEOID10')
df_census.shape

In [ ]:
# merge tract geometries with census variables and create med home value 1000s
tracts = tracts_ca.merge(df_census, left_index=True, right_index=True, how='left')
tracts['med_home_value_k'] = tracts['med_home_value'] / 1000
tracts.shape

In [ ]:
# choose which variables to use as predictors
predictors = ['pct_white', 'pct_built_before_1940', 'med_rooms_per_home', 'pct_bachelors_degree']

# choose a response variable and drop any rows in which it is null
response = 'med_home_value_k'
tracts = tracts.dropna(subset=[response])
tracts.shape

Get our data into the right format for estimating our model on them:
  - the **design matrix** is a $n×k$ matrix of $n$ non-null observations on $k$ predictor variables.
  - the **response matrix** is a $n×1$ matrix of $n$ non-null observations on the response variable.

In [ ]:
# create design matrix of predictors (drop nulls) and response matrix
X = tracts[predictors].dropna()
Y = tracts.loc[X.index][[response]]

In [ ]:
# estimate linear regression model with OLS
ols = ps.model.spreg.OLS(y=Y.values,
                         x=X.values,
                         name_x=X.columns.tolist(),
                         name_y=response,
                         name_ds='tracts')
print(ols.summary)

### Types of spatially explicit models:

  - **Spatial heterogeneity**: account for systematic differences across space without explicitly modeling interdependency
    - *spatial fixed effects*: intercept varies for each spatial group
    - *spatial regimes*: intercept and coefficients vary for each spatial group
    - *geographically weighted regression*: model local relationships that vary across study area
  - **Spatial dependence**: model interdependencies between observations through space
    - *spatial lag model*: spatially-lagged endogenous variable added as predictor (because of endogeneity, cannot use OLS to estimate)
    - *spatial error model*: spatial effects in error term
    - *spatial combo model*: both lag and error
    
## 3. Spatial fixed effects

Intercept varies for each each spatial group. Use dummy variables to represent the groups (counties) into which our observations (tracts) are nested.

In [ ]:
# create a new dummy variable for each county, with 1 if tract is in this county and 0 if not
for county in tracts['COUNTYFP'].unique():
    new_col = f'dummy_county_{county}'
    tracts[new_col] = (tracts['COUNTYFP'] == county).astype(int)

In [ ]:
# remove one dummy from dummies to prevent perfect collinearity
# ie, a subset of predictors sums to 1 (which full set of dummies will do)
county_dummies = [f'dummy_county_{county}' for county in tracts['COUNTYFP'].unique()]
county_dummies = county_dummies[:-1]
county_dummies

In [ ]:
# create design matrix of predictors (drop nulls) and response matrix
X = tracts[predictors + county_dummies].dropna()
Y = tracts.loc[X.index][[response]]

In [ ]:
# estimate linear regression model with spatial fixed effects
ols = ps.model.spreg.OLS(y=Y.values,
                         x=X.values,
                         name_x=X.columns.tolist(),
                         name_y=response,
                         name_ds='tracts')
print(ols.summary)

## 4. Spatial regimes

Intercept and coefficients vary for each spatial group (aka, regime). Here, the regimes are our 3 counties and we use OLS for estimation, but you can also combine spatial regimes with spatial autoregressive models (the latter is introduced later).

In [ ]:
# create design matrix of predictors (drop nulls), response matrix, and regimes vector
X = tracts[predictors].dropna()
Y = tracts.loc[X.index][[response]]
regimes = tracts.loc[X.index]['COUNTYFP']
regimes.sample(5)

In [ ]:
# estimate spatial regimes model with OLS
olsr = ps.model.spreg.OLS_Regimes(y=Y.values,
                                  x=X.values,
                                  regimes=regimes.values,
                                  name_regimes='county',
                                  name_x=X.columns.tolist(),
                                  name_y=response,
                                  name_ds='tracts')
print(olsr.summary)

## 5. Geographically weighted regression

GWR allows us to further investigate how model parameters and performance may vary across the study area. It estimates the regression model for each observation's local neighborhood, then combines these estimates into a global model for the study area. A user-defined *bandwidth* determines how these local estimates are calculated.

In [ ]:
# helper function to generate colormaps for GWR plots
def get_cmap(values, cmap_name='coolwarm', n=256):
    import numpy as np
    from matplotlib.colors import LinearSegmentedColormap as lsc
    name = f'{cmap_name}_new'
    cmap = plt.cm.get_cmap(cmap_name)
    vmin = values.min()
    vmax = values.max()

    if vmax < 0:
        # if all values are negative, use the negative half of the colormap
        return lsc.from_list(name, cmap(np.linspace(0, 0.5, n)))
    elif vmin > 0:
        # if all values are positive use the positive half of the colormap
        return lsc.from_list(name, cmap(np.linspace(0.5, 1, n)))
    else:
        # otherwise there are positive and negative values so use zero as midpoint
        # and truncate the colormap such that if the original spans ± the greatest
        # absolute value, we only use colors from it spanning vmin to vmax
        abs_max = max(values.abs())
        start = (vmin + abs_max) / (abs_max * 2)
        stop = (vmax + abs_max) / (abs_max * 2)
        return lsc.from_list(name, cmap(np.linspace(start, stop, n)))

In [ ]:
%%time
# select a bandwidth value for our GWR model, given the data
centroids = tracts.loc[X.index].centroid
coords = list(zip(centroids.x, centroids.y))
bw = ps.model.mgwr.sel_bw.Sel_BW(coords, Y.values, X.values).search()

In [ ]:
# what is the selected bandwidth value?
bw

In [ ]:
%%time
# estimate the GWR model parameters
model = ps.model.mgwr.gwr.GWR(coords=coords,
                              y=Y.values,
                              X=X.values,
                              bw=bw)
gwr = model.fit()

In [ ]:
# inspect the results
gwr.summary()

In [ ]:
# a constant was added, so we'll add it to our predictors
cols = ['constant'] + predictors
cols

In [ ]:
# turn GWR local parameter estimates into a GeoDataFrame with tract geometries
params = pd.DataFrame(gwr.params, columns=cols, index=X.index)
params = tracts[['geometry']].merge(params, left_index=True, right_index=True, how='right')
params.head()

In [ ]:
# plot the spatial distribution of local parameter estimates
# set nrows, ncols to match your number of predictors!
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
for col, ax in zip(predictors, axes.flat):
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(col)
    gdf = params.dropna(subset=[col], axis='rows')
    ax = gdf.plot(ax=ax,
                  column=col,
                  cmap=get_cmap(gdf[col]),
                  legend=True,
                  legend_kwds={'shrink': 0.6})
fig.tight_layout()

In [ ]:
# turn GWR local t-values into a GeoDataFrame with tract geometries
# set t-values below significance threshold to zero then clip to ± 4
tvals = pd.DataFrame(gwr.filter_tvals(alpha=0.05), columns=cols, index=X.index).clip(-4, 4)
tvals = tracts[['geometry']].merge(tvals, left_index=True, right_index=True, how='right')

# plot the spatial distribution of local t-values
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
for col, ax in zip(predictors, axes.flat):
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(col)
    gdf = tvals.dropna(subset=[col], axis='rows')
    ax = gdf.plot(ax=ax,
                  column=col,
                  cmap=get_cmap(gdf[col]),
                  legend=True,
                  legend_kwds={'shrink': 0.6})
fig.tight_layout()

In [ ]:
# turn GWR local R^2 values into a GeoDataFrame with tract geometries
col = 'Local R-squared'
r_squared = pd.DataFrame(gwr.localR2, index=X.index, columns=[col])
r_squared = tracts[['geometry']].merge(r_squared, left_index=True, right_index=True, how='right')

# plot the spatial distribution of local R^2
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal')
ax.axis('off')
ax.set_title(col)
gdf = r_squared.dropna(subset=[col], axis='rows')
ax = gdf.plot(ax=ax,
              column=col,
              cmap=get_cmap(gdf[col]),
              legend=True,
              legend_kwds={'shrink': 0.6})
fig.tight_layout()

## 6. Spatial diagnostics

So far we've seen different spatial heterogeneity models. Now we'll explore spatial dependence (modeling interdependencies between observations over space), starting by using queen-contiguity spatial weights to model spatial relationships between observations and OLS to check diagnostics.

In [ ]:
# compute spatial weights from tract geometries (but only those tracts that appear in design matrix!)
W = ps.lib.weights.Queen.from_dataframe(tracts.loc[X.index])
W.transform = 'r'

In [ ]:
# compute OLS spatial diagnostics to check the nature of spatial dependence
ols = ps.model.spreg.OLS(y=Y.values,
                         x=X.values,
                         w=W,
                         spat_diag=True,
                         moran=True)

In [ ]:
# calculate moran's I (for the response) and its significance
mi = ps.explore.esda.Moran(y=Y, w=W, two_tailed=True)
print(mi.I)
print(mi.p_sim)

In [ ]:
# moran's I (for the residuals): moran's i, standardized i, p-value
ols.moran_res

#### Interpreting the results

A significant Moran's *I* suggests spatial autocorrelation, but doesn't tell us which alternative specification should be used. Lagrange Multiplier (LM) diagnostics can help with that. If one LM test is significant and the other isn't, then that tells us which model specification (spatial lag vs spatial error) to use:

In [ ]:
# lagrange multiplier test for spatial lag model: stat, p
ols.lm_lag

In [ ]:
# lagrange multiplier test for spatial error model: stat, p
ols.lm_error

#### Interpreting the results

If (and only if) both the LM tests produce significant statistics, try the robust versions (the nonrobust LM tests are sensitive to each other):

In [ ]:
# robust lagrange multiplier test for spatial lag model: stat, p
ols.rlm_lag

In [ ]:
# robust lagrange multiplier test for spatial error model: stat, p
ols.rlm_error

So... which model specification to choose? Workflow:

  1. If neither LM test is significant: use regular OLS.
  2. If only one LM test is significant: use that model spec.
  3. If both LM tests are significant: run robust versions.
  4. If only one robust LM test is significant: use that model spec.
  5. If both robust LM tests are significant (this can often happen with large sample sizes):
     1. first consider if the initial model specification is actually a good fit
     2. if so, use the spatial specification corresponding to the larger robust-LM statistic
     3. or consider a combo model

## 7. Spatial lag model

When the diagnostics indicate the presence of a spatial diffusion process. Uses the spatially-lagged endogenous variable as a predictor. Because of endogeneity, cannot use OLS to estimate.

Model specification:

$y = \rho W y + \beta X + u$

where $y$ is a $n \times 1$ matrix of observations (response), $W$ is a $n \times n$ spatial weights matrix (thus $Wy$ is the spatially-lagged response), $\rho$ is the spatial autoregressive parameter to be estimated, $X$ is a $n \times k$ matrix of observations (exogenous predictors), $\beta$ is a $k \times 1$ vector of parameters (coefficients) to be estimated, and $u$ is a $n \times 1$ vector of errors.

In [ ]:
# maximum-likelihood estimation with full matrix expression
mll = ps.model.spreg.ML_Lag(y=Y.values,
                            x=X.values,
                            w=W,
                            method='full',
                            name_w='queen',
                            name_x=X.columns.tolist(),
                            name_y=response,
                            name_ds='tracts')
print(mll.summary)

In [ ]:
# the spatial autoregressive parameter estimate, rho
mll.rho

## 8. Spatial error model

When the diagnostics indicate the presence of spatial error dependence (spatial effects in error term).

Model specification:

$y = \beta X + u$

where $X$ is a $n \times k$ matrix of observations (exogenous predictors), $\beta$ is a $k \times 1$ vector of parameters (coefficients) to be estimated, and $u$ is a $n \times 1$ vector of errors. The errors $u$ follow a spatial autoregressive specification:

$u = \lambda Wu + \epsilon$

where $\lambda$ is a spatial autoregressive parameter to be estimated and $\epsilon$ is the vector of errors.

In [ ]:
# maximum-likelihood estimation with full matrix expression
mle = ps.model.spreg.ML_Error(y=Y.values,
                              x=X.values,
                              w=W,
                              method='full',
                              name_w='queen',
                              name_x=X.columns.tolist(),
                              name_y=response,
                              name_ds='tracts')
print(mle.summary)

In [ ]:
# the spatial autoregressive parameter estimate, lambda
mle.lam

## 9. Spatial lag+error combo model

Estimated with GMM (generalized method of moments). Essentially a spatial error model with endogenous explanatory variables.

Model specification:

$y = \rho W y + \beta X + u$

where $y$ is a $n \times 1$ matrix of observations (response), $W$ is a $n \times n$ spatial weights matrix (thus $Wy$ is the spatially-lagged response), $\rho$ is the spatial autoregressive parameter to be estimated, $X$ is a $n \times k$ matrix of observations (exogenous predictors), $\beta$ is a $k \times 1$ vector of parameters (coefficients) to be estimated, and $u$ is a $n \times 1$ vector of errors.

The errors $u$ follow a spatial autoregressive specification:

$u = \lambda Wu + \epsilon$

where $\lambda$ is a spatial autoregressive parameter to be estimated and $\epsilon$ is the vector of errors.

In [ ]:
gmc = ps.model.spreg.GM_Combo_Het(y=Y.values,
                                  x=X.values,
                                  w=W,
                                  name_w='queen',
                                  name_ds='tracts',
                                  name_x=X.columns.tolist(),
                                  name_y=response)
print(gmc.summary)

In [ ]:
# now it's your turn
# with a new set of predictors, compute spatial diagnostics and estimate a new spatial model accordingly
